<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/13_rnn/RNN_bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural NetworkによるBitcoinの価格予測

---
## 目的
Recurrent Neural Networkを使ってBitcoinの価格予測を行う．
ここで，今回はRecurrent Neural Networkの一種である，Long Short Term Memory（LSTM）を使用する．
また，PyTorchで使用されるデータセットオブジェクトの作成を行う．

## リカレントニューラルネットワーク
リカレントニューラルネットワークは，系列データを扱うことができるネットワークです．
例えば，「今日は良い天気です」という文章において，「今日は」，「良い」という時系列データを与えると，次に現れる単語として「天気」を予測するという問題です．
リカレントニューラルネットワークを利用することで，過去の系列情報から文脈の流れを考慮した予測ができるようになります．
応用例として，30分後の電力を予測する，翌日の株価を予測するなどの予測モデル，音声認識や機械翻訳などがあります．

## リカレントニューラルネットワークの種類
リカレントニューラルネットワークにはいくつかの種類があります．

* Elman Network：一般的なリカレントニューラルネットワーク．１時刻前の情報を内部状態として，現時刻の入力と合わせて中間層に与える
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/RNN.png?raw=true" width = 500>
* Jordan Network：１時刻前の出力層の情報を現時刻の入力と合わせて中間層に与える
* Echo state network (ESN)：一部の重みを乱数で初期化し，更新しない．中間層内のユニットは相互結合する
* Long Short-Term Memory (LSTM)：内部情報を記憶するメモリセルを持ち，複数のゲートによってメモリセルの情報を書き換えたり出力したりする
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/LSTM.png?raw=true" width = 500>
* Gated Recurrent Unit (GRU)：内部情報の保持方法をLSTMよりもシンプルな構造にしたリカレントニューラルネットワーク
<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/GRU.png?raw=true" width = 500>
* Bidirectional RNN：過去の情報だけでなく，未来の情報も利用する双方向のリカレントニューラルネットワーク

## リカレントニューラルネットワークの学習
リカレントニューラルネットワークは，時系列データを逐次与えます．
この流れを展開するとニューラルネットワークを時間方向につなげた大きなネットワークとみなすことができます．
そのため，リカレントニューラルネットワークの学習にもニューラルネットワークと同様に誤差逆伝播法を用いることができます．
リカレントニューラルネットワークでの誤差逆伝播法は， Back-propagation through time (BPTT)法と呼ばれています．

まず，図の黒矢印に従い，系列データを時刻$t=0$から順伝播します．
ネットワークは時刻ごとに別々にあるのではなく，１つのネットワークに対して逐次データを入力します．
その時，各時刻における各層の値は変わっていくので，それらを記憶しておきます．
また，順伝播時に各時刻における誤差を算出しておきます．

時刻$t=T$まで系列データの順伝播が終わると学習開始となります．
学習は誤差逆伝播法と同様に，BPTTでも誤差の勾配を求めて結合重みを更新します．
その際，時刻をさかのぼるように，時刻$t=T$の出力層から始めます．
学習では，以下の3箇所の結合重みを順番に更新します．
* 時刻tの出力層から時刻tの中間層間の結合重み
* 時刻tの中間層から時刻t-1の中間層間の結合重み
* 時刻tの中間層から時刻tの入力層間の結合重み

<img src="https://github.com/himidev/Lecture/blob/main/13_rnn/01_03_RNN/back.png?raw=true" width = 500>

## 0. データのダウンロード

プログラムに必要なデータをダウンロードします．

今回使用するデータは，Kaggleで公開されているBitcoinの価格を予測するデータセットです．

https://www.kaggle.com/datasets/team-ai/bitcoin-price-prediction

**今回使用するデータは，CSVファイルなどのテキストファイルで整理・保存されたデータを扱うための練習として採用しました．実際の現場などでは，センサーの数値データなどをCSVファイルに保存しておき，その数値データをもとに学習するような状況を想定しています．**





In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1_Gdneij6TP6CK_HCommCtbaitVfoY-fN', 'BitcoinPricePrediction.zip', quiet=False)
!unzip -q -o BitcoinPricePrediction.zip

ここで，一度データセットを確認してみましょう．

データ（フォルダ）を確認すると，BitcoinPricePredictionフォルダの中にTraining.csvとTest.csvという二つのCSVファイルが保存されています．

![BitcoinDataDir.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/e81aa369-427e-fe22-03cf-03cd82c917ce.png)

### CSVファイルの中身

それぞれの中身を見ると，
* Date: 日付
* Open: 始値
* High: 最高値
* Low: 最安値
* Close: 終値
* Volume: 取引ボリューム（取引数量）
* Market Cap: 時価総額
という列があり，それぞれの日付で値を持っていることがわかります．

また，Dateの値を確認すると，新 --> 古の順番に日付が並んでいることがわかります．

今回は，「Open, High, Low, Close」の値から翌日の「High, Low」を予測する再帰型ニューラルネットワークを構築して学習してみましょう．

## 1. モジュールのインポート部分

はじめに必要なモジュールをインポートする．

In [ ]:
import os
import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import csv

import matplotlib.pyplot as plt

## 2. データセットクラスの作成

ここでは，ダウンロードしたCSVファイルの形式に合わせて，PyTorchのデータセットクラスを自作します．

In [ ]:
class BitcoinPriceDataset(Dataset):

  def __init__(self, csv_file_path, time_window=10, max_price=None):
    super().__init__()

    self.csv_file_path = csv_file_path
    self.time_window = time_window

    ### csvファイルの読み込み
    with open(self.csv_file_path, 'r') as file:
      reader = csv.reader(file)

      ### 1行ずつデータをリストへ追加
      csv_data_source = []
      for row in reader:
        csv_data_source.append(row)

    ### ヘッダー行の保存（使用しないかもしれません）
    self.header = csv_data_source[0]
    ### データ行の保存（ヘッダ以外の行を保存）
    self.csv_data = csv_data_source[1:]

    ### 総データ数の保存
    self.num_data = len(self.csv_data)

    ### 日付データの保存（使用しないかもしれません）
    self.date = []
    for row in self.csv_data:
      self.date.append(row[0])

    ### 数値データの保存
    self.bitcoin_data = torch.zeros([self.num_data, 4], dtype=torch.float32)
    for i, row in enumerate(self.csv_data):
      self.bitcoin_data[i, 0] = float(row[1])
      self.bitcoin_data[i, 1] = float(row[2])
      self.bitcoin_data[i, 2] = float(row[3])
      self.bitcoin_data[i, 3] = float(row[4])

    ### データの順番を入れ替え（新~旧 --> 旧~新）
    self.date.reverse()
    self.bitcoin_data = torch.flipud(self.bitcoin_data)

    ### 最大の価格値
    if max_price is None:
      self.max_price = torch.max(self.bitcoin_data)
    else:
      self.max_price = max_price

    ### 数値データの正規化（0.0 ~ 1.0）
    self.bitcoin_data /= self.max_price

  def __getitem__(self, item):

    input = self.bitcoin_data[item:item+self.time_window, :]
    output = self.bitcoin_data[item+1:item+self.time_window+1, 1:3]

    return input, output

  def __len__(self):
    return self.num_data - self.time_window

## 3. ネットワークモデルの定義

続いてネットワークを定義します．

今回は再帰型ニューラルネットワークを定義します．




この時，各レイヤーのクラスの詳細などを調べたい場合は，PyTorchのReferenceを参照しつつ実装します．

[PyTorch reference mannual](https://pytorch.org/docs/stable/index.html)



In [ ]:
class MyLSTM(nn.Module):

  ### ネットワーク構造
  # RNN (LSTM) --> 全結合層 --> 予測結果

  def __init__(self, in_size=4, out_size=2, hidden_size=32):
    super().__init__()

    ### RNN (LSTM) 層の定義
    self.recurrent = nn.LSTMCell(input_size=in_size, hidden_size=hidden_size)

    ### 全結合層の定義
    self.fc = nn.Linear(in_features=hidden_size, out_features=out_size)

  def forward(self, x, hx, cx):
    hx, cx = self.recurrent(x, (hx, cx))
    h = self.fc(hx)
    return h, hx, cx

## 4. 学習の準備

ここでは，学習に必要な

* ネットワークモデル
* 誤差関数
* 最適化手法
* データセット

の定義を行います．

最適化関数などもReference mannualを参照しつつ好きなものを選択記述しましょう．

**DataLoaderのnum_workersについて**

`torch.utils.data.DataLoader`の引数である`num_workers`は，データを読み込んで準備する処理を並列処理するための引数です．例えば，`num_workers=10`とした場合には，10並列でデータの読込処理 (データセットクラスの`__getitem__()`) を10並列で実行してくれます．そのため，使用する計算機のCPU性能に合わせて，ある程度大きな数を指定しておくとデータの読込処理が早くなり，学習の高速化が期待できます．

In [ ]:
### GPUが使えるかどうか
use_cuda = torch.cuda.is_available()
print("Use CUDA:", use_cuda)

### ネットワークモデル
n_hidden = 32
model = MyLSTM(in_size=4, out_size=2, hidden_size=n_hidden)
if use_cuda:
  model = model.cuda()

### 誤差関数
criterion = nn.MSELoss()
if use_cuda:
  criterion = criterion.cuda()

### 最適化関数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### データセットクラス
time_window = 10
train_dataset = BitcoinPriceDataset(csv_file_path="BitcoinPricePrediction/Training.csv", time_window=time_window, max_price=None)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

### 学習データ内の最大の価格の値を保存 (後程テストに使用します)
train_max_price = train_dataset.max_price

 ## 5. 学習の開始

 上で定義したモデルや最適化手法，データセットを用いて学習を行います．

In [ ]:
### Epoch数などの指定
num_epochs = 20

### ネットワークを学習モードへ変更
model.train()

### 学習経過を保存するためのリストを用意
loss_list = []

### 学習ループ (for文)
print("training; start ---------------------")
for epoch in range(1, num_epochs+1):
  print("Epoch:", epoch)

  # 1 epochごとの学習経過を計算するための変数を用意
  loss_sum = 0.0

  for input, label in train_loader:
    # 隠れ状態，セル状態の変数の初期化
    hx = torch.zeros(input.size(0), n_hidden)
    cx = torch.zeros(input.size(0), n_hidden)

    if use_cuda:
      input = input.cuda()
      label = label.cuda()
      hx = hx.cuda()
      cx = cx.cuda()

    loss = 0.0
    for time_index in range(input.size(1)):
      y, hx, cx = model(input[:, time_index, :], hx, cx)
      loss += criterion(y, label[:, time_index, :])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # lossを加算（学習経過確認用）
    loss_sum += loss.item()

  ### epochが終了したタイミングでそのエポックの平均誤差を表示（1 epoch内のiterationとtime_windowで割った値）
  print("  Loss:", loss_sum / len(train_loader) / time_window)

  ### 上で表示した数値をリストへ保存
  loss_list.append(loss_sum / len(train_loader) / time_window)

print("training; done ----------------------")

## 6. 評価

学習したモデルを用いて評価を行います．

### モデルの読み込み

学習中に保存したモデルを読み込みます．

In [ ]:
### ネットワークモデルの定義（この時点ではパラメータはランダム）
model = MyLSTM(in_size=4, out_size=2, hidden_size=n_hidden)
if use_cuda:
  model = model.cuda()

### 学習したモデルパラメータの読み込み
trained_parameter = torch.load("checkpoint-0020.pt")
model.load_state_dict(trained_parameter)

### 推論と結果の表示

読み込んだ学習済みモデルを用いて予測を行います．

今回のネットワークは，数値データ（価格）の予測値を出力するモデルのため，誤差ではなく，その予測値をリストに保存して，後程グラフ表示をして結果を確認します．


**学習データに対する予測**

まずは，学習に使用したデータでどの程度予測できるかを確認します．

In [ ]:
pred = []
true = []

train_dataset_eval = BitcoinPriceDataset(csv_file_path="BitcoinPricePrediction/Training.csv", time_window=1, max_price=train_max_price)
train_loader_eval = torch.utils.data.DataLoader(train_dataset_eval, batch_size=1, shuffle=False, num_workers=2)

model.eval()

with torch.no_grad():

  # 隠れ状態，セル状態の変数の初期化
  hx = torch.zeros(1, n_hidden)
  cx = torch.zeros(1, n_hidden)
  if use_cuda:
    hx = hx.cuda()
    cx = cx.cuda()

  for input, label in train_loader_eval:

    if use_cuda:
      input = input.cuda()
      label = label.cuda()

    output, hx, cx = model(input[:, 0, :], hx, cx)

    pred.append(output.tolist())
    true.append(label.tolist())

### 保存した正解・予測結果（リスト形式）をtorch.tensor形式に変換
pred_tensor = torch.tensor(pred).squeeze()
true_tensor = torch.tensor(true).squeeze()

### グラフの横軸用のリストを用意
time_index = list(range( pred_tensor.size(0) ))
print("time index list:", time_index)

### グラフの描画・表示・保存
plt.plot(time_index, pred_tensor[:, 0], '-b', label='high pred')
plt.plot(time_index, true_tensor[:, 0], '-r', label='high true')
plt.plot(time_index, pred_tensor[:, 1], '-c', label='low pred')
plt.plot(time_index, true_tensor[:, 1], '-y', label='low true')
plt.xlabel("day")
plt.ylabel("price")
plt.title("Prediction Results for Training Data")
plt.legend()
plt.savefig("prediction_train.pdf")
plt.show()
plt.clf()

**テストデータに対する予測**

次に，テスト用データでどの程度予測できるかを確認します．

In [ ]:
pred = []
true = []

test_dataset = BitcoinPriceDataset(csv_file_path="BitcoinPricePrediction/Test.csv", time_window=1, max_price=train_max_price)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

model.eval()

with torch.no_grad():

  # 隠れ状態，セル状態の変数の初期化
  hx = torch.zeros(1, n_hidden)
  cx = torch.zeros(1, n_hidden)

  if use_cuda:
    hx = hx.cuda()
    cx = cx.cuda()


  for input, label in test_loader:

    if use_cuda:
      input = input.cuda()
      label = label.cuda()

    output, hx, cx = model(input[:, 0, :], hx, cx)

    pred.append(output.tolist())
    true.append(label.tolist())


### 保存した正解・予測結果（リスト形式）をtorch.tensor形式に変換
pred_tensor = torch.tensor(pred).squeeze()
true_tensor = torch.tensor(true).squeeze()

### グラフの横軸用のリストを用意
time_index = list(range( pred_tensor.size(0) ))
print("time index list:", time_index)

### グラフの描画・表示・保存
plt.plot(time_index, pred_tensor[:, 0], '-b', label='high pred')
plt.plot(time_index, true_tensor[:, 0], '-r', label='high true')
plt.plot(time_index, pred_tensor[:, 1], '-c', label='low pred')
plt.plot(time_index, true_tensor[:, 1], '-y', label='low true')
plt.xlabel("day")
plt.ylabel("price")
plt.title("Prediction Results for Test Data")
plt.legend()
plt.savefig("prediction_test.pdf")
plt.show()
plt.clf()